# Regionalizing supply chains

Kernel: `bw25`

In [ ]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [ ]:
if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")

In [ ]:
bd.projects.set_current('ei38-teaching-25')

We want to build a motor in S Korea - currently we only have a global one.

In [ ]:
motor = bd.get_activity(database="ei 3.8 cutoff", name="electric motor production, vehicle (electric powertrain)")
motor

Let's figure out the most important exchanges in the current activity:

In [ ]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [ ]:
ba.print_recursive_calculation(motor, ipcc, cutoff=0.025)

Aluinium and steel seem important. Let's see if we have the potential to disaggregate the aluminium production (if there aren't separate ecoinvent activities, then getting trade data doesn't help...).

In [ ]:
ba.print_recursive_supply_chain(bd.get_activity(database="ei 3.8 cutoff", name="market for aluminium, wrought alloy"))

Yes, we can disaggregate the `'aluminium ingot, primary, to aluminium, wrought alloy market'`. The key to me is:

     0.696: 'aluminium ingot, primary, to aluminium, wrought alloy market' (kilogr
        0.541: 'market for aluminium, primary, ingot' (kilogram, RoW, None)
        0.057: 'market for aluminium, primary, ingot' (kilogram, IAI Area, North Amer
        0.0982: 'market for aluminium, primary, ingot' (kilogram, IAI Area, EU27 & EFT

Each of these markets *could* include more specific suppliers.

As this product is deep in the supply chain, we could create a copy of each link down to the wrought aluminium input. However, that seems like a lot of work; instead, how about we create a "patch" dataset which *subtracts* the current aluminium mix *from the motor*, and adds in our new mix of aluminium production. In this case, we won't have a correct model of the supply chain (as we will be patching the motor production), but we spare some work.

We can read the above and see that we need 0.1685 kilograms of primary alimuinium production. Let's see what countries or regions produce primary aluminium:

In [ ]:
[x for x in bd.Database("ei 3.8 cutoff") if x['name'] == 'aluminium production, primary, ingot']

OK, a bit annoying, there are [International Aluminium Institute](https://international-aluminium.org/) regions.

Now we need some data on the specific aluminium imports of Korea. Ideally we would have this data for the specific industry of motor production, but for now we take something more general: EXIOBASE.

In [ ]:
if "EXIOBASE 3.8.1 2017 monetary" not in bd.databases:
    bi.exiobase_monetary()

In [ ]:
exio = bd.Database("EXIOBASE 3.8.1 2017 monetary")

Get list of activities in Korea

In [ ]:
sorted([ds for ds in exio if ds['location'] == 'KR'], key=lambda x: x['name'])

In [ ]:
alu = bd.get_activity(database="EXIOBASE 3.8.1 2017 monetary", name='Aluminium production', location='KR')

Aluminium importers

In [ ]:
for exc in filter(
        lambda exc: (exc.input['name'] == exc.output['name']) & (exc['amount'] > 0.005), 
        alu.technosphere()):
    print(exc)

This data is monetary, but we assume that as a globally traded commodity, the price roughly corresponds to the mass. This is not perfect! To be more specific one could use e.g. the BACI trade database which has much more specific goods classifiers.

Manually construct mapping between ecoinvent spatial scale and exiobase spatial scale:

In [ ]:
mapping = {
    "US": "RoW",
    "JP": "IAI Area, Asia, without China and GCC",
    "CN": "CN",
    "CA": "CA",
    "KR": "IAI Area, Asia, without China and GCC",
    "IN": "IAI Area, Asia, without China and GCC",
    "RU": "IAI Area, Russia & RER w/o EU27 & EFTA",
    "AU": "UN-OCEANIA",
    "WA": "IAI Area, Asia, without China and GCC",
    "WM": "IAI Area, Gulf Cooperation Council",
}

Aggregate the exiobase data to create our Korean aluminium market mix:

In [ ]:
from collections import defaultdict

market = defaultdict(float)

for exc in filter(
        lambda exc: (exc.input['name'] == exc.output['name']) & (exc['amount'] > 0.005), 
        alu.technosphere()):
    market[bd.get_activity(
        database="ei 3.8 cutoff",
        name="aluminium production, primary, ingot",
        location=mapping[exc.input['location']]
    )] += exc['amount']

Normalize to create a market with a total of 1 kilogram of production

In [ ]:
total = sum(market.values())

market = {key: value / total for key, value in market.items()}
market

OK, so we are almost done. We have 0.1309 kilograms of aluminium to substitute. In a real research project you should do this in a new `Database` and `Activity` (or activities); for now, we can just create a temporary datapackage.

To avoid changing the global motor production, we will create a new activity node, but one which doesn't have an ID in the relational database. This can be any integer, but shouldn't overlap with the ids we already have. 1.000.000 should be safe enough :)

In [ ]:
alu_demanded = 0.1685
gangnam_style = 1_000_000

indices = np.array(
    [
        (gangnam_style, gangnam_style), # Production exchange for new motor
        (motor.id, gangnam_style),  # Need the motor
        (bd.get_activity(database="ei 3.8 cutoff", name='aluminium ingot, primary, to aluminium, wrought alloy market').id, gangnam_style),
    ] + [
        (node.id, gangnam_style) for node in market
    ], dtype=bwp.INDICES_DTYPE
)
data = np.array([
        1,
        1,
        alu_demanded,
    ] + [
        value * alu_demanded for value in market.values()
    ]
) 
flip = np.array(
    [False, True, False] + [True for _ in market]
)

In [ ]:
dp = bwp.create_datapackage()

dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip_array=flip,
    name="Korean motor",
)

In [ ]:
_, data_objs, _ = bd.prepare_lca_inputs({motor: 1}, ipcc)

In [ ]:
lca = bc.LCA({motor.id: 1}, data_objs=data_objs + [dp])
lca.lci()
lca.lcia()
lca.score

In [ ]:
lca.lcia({gangnam_style: 1})
lca.score

## Challenges:

* There is also some recycled aluminium. Can you change the electricity used in this recycling to come from the Korean market mix?
* Copper is also important. Could you combine this notebook with notebook 1 and come up with a Korea-specific copper mix for motor production?